In [1]:
import os.path
import pandas as pd

In [2]:
wa_ev_df = pd.read_csv('data/Washington_EV_Population_Data.csv')

In [3]:
len(wa_ev_df)

91933

In [4]:
### Data Pre-processing and Cleaning

In [5]:
def download_file(file_name: str): # wa_ev_registrations_public
    if not os.path.exists('data/{}.csv'.format(file_name)):
        import requests
        url = "https://www.atlasevhub.com/public/dmv/{}.csv".format(file_name)
        file = requests.get(url)
        print(file)
        with open('data/{}.csv'.format(file_name), 'wb') as text:
            text.write(file.content)

download_file('wa_ev_registrations_public')

<Response [200]>


In [28]:
wa_df = pd.read_csv('data/wa_ev_registrations_public.csv') # , nrows=20000
print(wa_df.dtypes)
wa_df['Valid Date'] = pd.to_datetime(wa_df['Registration Valid Date'])
print(wa_df.dtypes)

Vehicle ID                       object
ZIP Code                          int64
Registration Valid Date          object
VIN Prefix                       object
VIN Model Year                   object
DMV ID                            int64
DMV ID Complete                  object
DMV Snapshot                     object
Registration Expiration Date    float64
State Abbreviation               object
Geography                        object
Vehicle Name                     object
Technology                       object
dtype: object


KeyboardInterrupt: 

In [27]:
# show the earliest and latest registration date, which also represents the data capture time period
min(wa_df['Valid Date']), max(wa_df['Valid Date'])

(Timestamp('2010-01-06 00:00:00'), Timestamp('2022-03-16 00:00:00'))